## Chapter 01. Language Model

### Language Model

#### 1. Vector

In [34]:
import numpy as np
a = np.array([1, 2, 3]) # 리스트 형태로 벡터 생성, 파이썬의 리스트를 numpy의 array(배열)로 변환
a # numpy 내부적으로 벡터나 행렬을 효율적으로 처리하기 위해 만든 자료 구조

array([1, 2, 3])

In [35]:
b = np.array([5, 6, 7])

In [36]:
a * b # 단순 곱 = 배열 위치에 따른 짝지은 곱

array([ 5, 12, 21])

In [37]:
np.dot(a, b) # dot product, 곱한 수의 합계

38

In [38]:
a @ b # 새로운 연산자(dot product)

38

In [39]:
np.einsum('i,i', a, b) # einstein sum

38

#### 2. Matrix

In [40]:
c = np.array([[1, 2, 3], [4, 5, 6]]) # 리스트 형태로 i번째 행을 표현, 하나의 행을 리스트로 표현
c

array([[1, 2, 3],
       [4, 5, 6]])

In [41]:
d = np.array([[1, 2], [3, 4], [5, 6]]) # c가 3열이므로, d는 무조건 3행이 와야함
d

array([[1, 2],
       [3, 4],
       [5, 6]])

In [42]:
np.dot(c, d)

array([[22, 28],
       [49, 64]])

In [43]:
c @ d

array([[22, 28],
       [49, 64]])

In [44]:
c[0, :] # 첫 번째 행

array([1, 2, 3])

In [45]:
d[:, 0] # 첫 번째 열

array([1, 3, 5])

In [47]:
c[0, :] @ d[:, 0] # 첫 번째 행과 첫 번째 열의 곱, 1k와 k1의 곱

22

In [48]:
np.einsum('ik,kj->ij', c, d)

array([[22, 28],
       [49, 64]])

### Transformers

In [ ]:
# !pip install transformers

In [49]:
from transformers import pipeline # 특정한 종류의 자연어 처리 작업을 간단히 할 수 있도록 지원
classifier = pipeline('sentiment-analysis') 
# 트랜스포머 모형을 이용한 감성분석, 하고자하는 과제의 종류를 파이프라인에 넣어 과제에 맞는 학습된 모형을 다운받음

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

c:\Users\lucky\anaconda3\envs\TEST\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\lucky\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [50]:
classifier('I am glad to hear that you finally made it.') #감성분석: 긍정

[{'label': 'POSITIVE', 'score': 0.9998080134391785}]

In [51]:
classifier('I am sorry that you hate it.') #감성분석: 부정

[{'label': 'NEGATIVE', 'score': 0.9981151819229126}]

In [52]:
classifier = pipeline('sentiment-analysis', model="monologg/koelectra-small-finetuned-nsmc") # 한국어 가능 모델 적용
# 한국어 영화평을 기준으로 감성분석이 실행되어 있는 모델 

config.json:   0%|          | 0.00/756 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/55.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/102 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/247k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [53]:
classifier('이 영화 진짜 재밌다')

[{'label': 'positive', 'score': 0.9803304076194763}]

In [54]:
classifier('이야기가 말이 안된다')

[{'label': 'negative', 'score': 0.8447047472000122}]

### Tokenizers

In [ ]:
# !pip install tokenizers

In [ ]:
# !wget -c https://github.com/songys/Chatbot_data/raw/master/ChatbotData%20.csv -O chatbot.csv

In [3]:
import pandas as pd
df = pd.read_csv('./data/chatbot.csv')
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [10]:
with open('./data/sample.txt','w',encoding='utf-8') as f: # sample.txt 라는 이름으로 저장
  for row in df.itertuples():
    f.write(row.Q + '\n')
    f.write(row.A + '\n')

In [ ]:
# !head sample.txt

In [11]:
from tokenizers import CharBPETokenizer # 글자 단위의 BPE 토큰화
bpe = CharBPETokenizer(lowercase=True) # 대문자를 소문자로 바꾸는 옵션

In [13]:
# 어휘를 학습하는 함수 
# 파일이 여러 개라면 리스트 형태로 제시 
# min_frequency = 최소 몇 번 나와야 어휘에 포함
# vocab_size = 몇 개의 어휘를 학습할 것인가
bpe.train(files='./data/sample.txt',min_frequency=1, vocab_size=5000) 

In [14]:
enc = bpe.encode('자연어 처리는 재밌다!')
enc.ids #텍스트를 토큰화하여 고유번호로 변환

[2203, 1016, 797, 1872, 2248, 1018, 1484]

In [15]:
enc.tokens #고유번호의 글자 확인

['자연', '어</w>', '처', '리는</w>', '재밌', '다</w>', '!</w>']

In [16]:
import unicodedata
with open('decomposed.txt','w',encoding='utf-8') as f:
  for row in df.itertuples():
    q = unicodedata.normalize('NFD', row.Q)
    f.write(q + '\n')
    a = unicodedata.normalize('NFD', row.A)
    f.write(a + '\n')

In [ ]:
# !head decomposed.txt 
# 윈도우 메모장에선 풀어쓰기 되어있음

In [17]:
bpe = CharBPETokenizer(lowercase=True)
bpe.train(files='decomposed.txt',min_frequency=1,vocab_size=5000)

In [18]:
text = unicodedata.normalize('NFD','자연어 처리는 재밌다!') # 풀어쓰기로 학습시켰으므로, 토큰화 시킬 때도 풀어쓰기해줘야함
enc = bpe.encode(text)

In [19]:
enc.ids

[1290, 204, 299, 1547, 819, 1365, 152]

In [20]:
enc.tokens

['자연', '어</w>', '처', '리는</w>', '재미', 'ᆻ다</w>', '!</w>']

In [22]:
from tokenizers import ByteLevelBPETokenizer #바이트단위 토큰화
byte = ByteLevelBPETokenizer(lowercase=True)
byte.train(files='./data/sample.txt', min_frequency=1, vocab_size=5000)

In [23]:
enc = byte.encode('자연어 처리는 재밌다!')
enc.ids

[2500, 273, 3488, 99, 1056, 2276, 294, 0]

In [24]:
enc.tokens # 한글은 여러 개의 바이트로 되어있기 때문에 바이트 단위로 쪼개지면서 한글이 아닌 것처럼 보임

['ìŀĲìĹ°', 'ìĸ´', 'Ġì²ĺë', '¦', '¬ëĬĶ', 'Ġìŀ¬ë°Į', 'ëĭ¤', '!']

In [25]:
from tokenizers import BertWordPieceTokenizer # wordpiece model
wp = BertWordPieceTokenizer(lowercase=True)

In [27]:
wp.train(files='./data/sample.txt',min_frequency=1,vocab_size=5000)
enc = wp.encode('자연어 처리는 재밌다!')

In [28]:
enc.ids

[1379, 201, 1014, 1028, 3349, 216, 5]

In [29]:
enc.tokens # ## <- 앞에 말에 이어서 쓰라는 의미, BPE는 끝나는 지점을 표시하고, WordPiece는 이어지는 부분을 표시

['자연', '##어', '처', '##리는', '재밌', '##다', '!']

In [31]:
from tokenizers import SentencePieceBPETokenizer
sp = SentencePieceBPETokenizer() # SentencePiece는 대문자를 소문자로 변화하는 옵션이 없음
sp.train('./data/sample.txt',min_frequency=1,vocab_size=5000)

In [32]:
enc= sp.encode('자연어 처리는 재밌다!')
enc.ids

[1888, 620, 1543, 1397, 1944, 226, 3]

In [33]:
enc.tokens

['▁자연', '어', '▁처', '리는', '▁재밌', '다', '!']